In [2]:
import time
import pyspark.sql.functions as f

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *

from pyspark.sql.functions import udf
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import desc

from itertools import islice




In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Assignment") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sqlSparkContext = SQLContext(sc)

In [4]:
df = sqlSparkContext.sql( \
'''
    SELECT 
        *,
        count(*) OVER (PARTITION BY session_id ORDER BY timestamp ASC) as rank
    FROM (
        SELECT 
            event_page,
            event_type,
            CAST(timestamp AS INT) AS timestamp,
            CAST((session_id || user_id) AS INT) AS session_id
        FROM kkingsep_final.clickstream
        WHERE 
            timestamp IS NOT NULL AND
            event_type != 'event'
        ) AS tbl_1
    ORDER BY
        session_id, timestamp
''')

In [ ]:
df.take(50)

In [5]:
def magic_func(df):
    result = {}
    dataCollect = df.collect()
    
    for row in dataCollect:
        session_id = row['session_id']
        event_type = row['event_type']
        event_page = row['event_page']

        if session_id not in result:
            result[session_id] = []

        if event_type == 'page':
            result[session_id].append(event_page)
        else:
            result[session_id].append('error')
    
    rows = []
    
    for k, v in result.items():
        # create route
        route = []
        for page in v:
            if page == 'error': break
            if len(route) > 0:
                if route[-1] == page:
                    continue
            route.append(page)

        route = '-'.join(route)
        
        # Add session_id, route to DataFrame
        rows.append(Row(k, route))
    
    df2 = spark.createDataFrame(rows, ['session_id', 'route'])
    
    return df2

In [6]:
df = magic_func(df)

In [9]:
sqlSparkContext.setConf("hive.exec.dynamic.partition", "true")
sqlSparkContext.setConf("hive.exec.dynamic.partition.mode", "nonstrict")

In [13]:
df.write.saveAsTable('clickstream_final')

In [20]:
df = sqlSparkContext.sql( \
'''
    SELECT 
        route, count(session_id) as cnt
    FROM clickstream_final
    GROUP BY route
    ORDER BY count(session_id) DESC
    LIMIT 30
''')

In [21]:
df.take(50)

[Row(route='main', cnt=39665),
 Row(route='main-tariffs', cnt=6601),
 Row(route='main-news', cnt=6335),
 Row(route='main-archive', cnt=5914),
 Row(route='main-family', cnt=4898),
 Row(route='main-digital', cnt=4267),
 Row(route='main-bonus', cnt=3529),
 Row(route='main-tariffs-news', cnt=1201),
 Row(route='main-news-tariffs', cnt=1147),
 Row(route='main-tariffs-archive', cnt=1049),
 Row(route='main-news-archive', cnt=1020),
 Row(route='main-archive-tariffs', cnt=1008),
 Row(route='main-archive-news', cnt=1005),
 Row(route='main-family-tariffs', cnt=925),
 Row(route='main-news-family', cnt=924),
 Row(route='main-tariffs-family', cnt=923),
 Row(route='main-family-news', cnt=886),
 Row(route='main-archive-family', cnt=821),
 Row(route='main-news-digital', cnt=803),
 Row(route='main-tariffs-main', cnt=791),
 Row(route='main-family-archive', cnt=777),
 Row(route='main-tariffs-digital', cnt=758),
 Row(route='main-digital-news', cnt=754),
 Row(route='main-digital-tariffs', cnt=729),
 Row(rout

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 53773)


Traceback (most recent call last):
  File "/usr/lib/python3.4/socketserver.py", line 305, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.4/socketserver.py", line 331, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.4/socketserver.py", line 344, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.4/socketserver.py", line 673, in __init__
    self.handle()
  File "/u0/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/u0/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError


----------------------------------------
